# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab2int = {word: idx for idx, word in enumerate(set(text))}
    int2vocab = {idx: word for word, idx in vocab2int.items()}
    
    # return tuple
    return (vocab2int, int2vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punc2token = {
        '.': '||Period||',
        ',': '||Comma||',
        '"': '||Quotation_Mark||',
        ';': '||Semicolon||',
        '!': '||Exclamation_Mark||',
        '?': '||Question_Mark||',
        '(': '||Left_Parentheses||',
        ')': '||Right_Parentheses||',
        '-': '||Dash||',
        '\n': '||Return||'
    }
        
    return punc2token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [3]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    sequences, labels = [], []
    for i in range(len(words)-sequence_length):
        sequences.append(words[i:i+sequence_length])
        labels.append(words[i+sequence_length])
    
    tensor_dataset = TensorDataset(torch.from_numpy(np.array(sequences)),
                                   torch.from_numpy(np.array(labels)))
    data_loader = DataLoader(tensor_dataset,
                             batch_size=batch_size,
                             shuffle=True)
    
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own

In [4]:
# data_loader = batch_data(int_text, 100, 30)
# iter(data_loader).next()

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [5]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 23,  24,  25,  26,  27],
        [ 41,  42,  43,  44,  45],
        [ 20,  21,  22,  23,  24],
        [ 27,  28,  29,  30,  31],
        [ 19,  20,  21,  22,  23],
        [ 18,  19,  20,  21,  22],
        [ 26,  27,  28,  29,  30],
        [ 38,  39,  40,  41,  42],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15]])

torch.Size([10])
tensor([ 28,  46,  25,  32,  24,  23,  31,  43,  39,  16])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        # define model layers
        self.in_embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=n_layers,
                            dropout=dropout,
                            batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim*3)
        self.fc2 = nn.Linear(hidden_dim, vocab_size)
        self.dropout = nn.Dropout(dropout)
        #self.out_embed = nn.Embedding(hidden_dim, vocab_size)
        
        # initialize weights to help with convergence
        self.in_embed.weight.data.uniform_(-1,1)
        #self.out_embed.weight.data.uniform_(-1,1)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)
        
        # embed in
        in_embeds = self.in_embed(nn_input.long())
        
        # lstsm
        lstm_out, hidden = self.lstm(in_embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # embed out
        #out_embeds = self.out_embed(lstm_out.long())
        #output = F.relu(self.fc1(lstm_out))
        #output = self.dropout(output)
        output = self.fc2(lstm_out)
        
        # retrieve last word score
        output = output.view(batch_size, -1, self.output_size)
        out = output[:, -1]

        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data
        
        # initialize hidden state with zero weights, and move to GPU if available
        if torch.cuda.is_available():
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [5]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    optimizer.zero_grad()
    hidden = tuple([each.data for each in hidden])
    
    # move to gpu if available
    if torch.cuda.is_available():
        inp, target = inp.cuda(), target.cuda()
    
    # perform backpropagation and optimization
    out, hidden = rnn.forward(inp, hidden)
    loss = criterion(out.squeeze(), target)
    loss.backward()
    
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []
        
        # save every epoch just in case
        torch.save(rnn.state_dict(), './model_epoch_point_3.pt')

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [7]:
import signal

from contextlib import contextmanager

import requests


DELAY = INTERVAL = 4 * 60  # interval time in seconds
MIN_DELAY = MIN_INTERVAL = 2 * 60
KEEPALIVE_URL = "https://nebula.udacity.com/api/v1/remote/keep-alive"
TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
TOKEN_HEADERS = {"Metadata-Flavor":"Google"}


def _request_handler(headers):
    def _handler(signum, frame):
        requests.request("POST", KEEPALIVE_URL, headers=headers)
    return _handler


@contextmanager
def active_session(delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import active session

    with active_session():
        # do long-running work here
    """
    token = requests.request("GET", TOKEN_URL, headers=TOKEN_HEADERS).text
    headers = {'Authorization': "STAR " + token}
    delay = max(delay, MIN_DELAY)
    interval = max(interval, MIN_INTERVAL)
    original_handler = signal.getsignal(signal.SIGALRM)
    try:
        signal.signal(signal.SIGALRM, _request_handler(headers))
        signal.setitimer(signal.ITIMER_REAL, delay, interval)
        yield
    finally:
        signal.signal(signal.SIGALRM, original_handler)
        signal.setitimer(signal.ITIMER_REAL, 0)


def keep_awake(iterable, delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import keep_awake

    for i in keep_awake(range(5)):
        # do iteration with lots of work here
    """
    with active_session(delay, interval): yield from iterable

In [8]:
# Data params
# Sequence Length
sequence_length = 10 # of words in a sequence
# Batch Size
batch_size = 32

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [9]:
# Training parameters
# Number of Epochs
num_epochs = 30
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 250
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 5000

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [14]:
torch.cuda.empty_cache()

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

with active_session():
    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('./save/final_trained_rnn', trained_rnn)
    print('Model Trained and Saved')

```
Training for 30 epoch(s)...
Epoch:    1/30    Loss: 4.872260116147995
Epoch:    1/30    Loss: 4.416109912395477
Epoch:    1/30    Loss: 4.283755551195145
Epoch:    1/30    Loss: 4.231456006431579
Epoch:    1/30    Loss: 4.209319970178604
Epoch:    2/30    Loss: 4.102239921117803
Epoch:    2/30    Loss: 4.0604443650245665
Epoch:    2/30    Loss: 4.070259440851212
Epoch:    2/30    Loss: 4.083727899694443
Epoch:    2/30    Loss: 4.07529057598114
Epoch:    3/30    Loss: 3.9789640582827452
Epoch:    3/30    Loss: 3.944721326494217
Epoch:    3/30    Loss: 3.989183581185341
Epoch:    3/30    Loss: 4.0089776851177215
Epoch:    3/30    Loss: 4.0039876540184025
Epoch:    4/30    Loss: 3.9206691014339974
Epoch:    4/30    Loss: 3.9003301032066346
Epoch:    4/30    Loss: 3.914500817346573
Epoch:    4/30    Loss: 3.9470286949157716
Epoch:    4/30    Loss: 3.960384136199951
Epoch:    5/30    Loss: 3.8721005740308554
Epoch:    5/30    Loss: 3.8403216766595842
Epoch:    5/30    Loss: 3.8703652626037597
Epoch:    5/30    Loss: 3.8891223197937013
Epoch:    5/30    Loss: 3.937563276171684
Epoch:    6/30    Loss: 3.842166624371247
Epoch:    6/30    Loss: 3.813496351695061
Epoch:    6/30    Loss: 3.8399237343311308
Epoch:    6/30    Loss: 3.8609102451086046
Epoch:    6/30    Loss: 3.905675974774361
Epoch:    7/30    Loss: 3.8101477290083534
Epoch:    7/30    Loss: 3.7735347113132476
Epoch:    7/30    Loss: 3.8345247956752777
Epoch:    7/30    Loss: 3.8478233501195906
Epoch:    7/30    Loss: 3.874689764690399
Epoch:    8/30    Loss: 3.7765360062392905
Epoch:    8/30    Loss: 3.783821085190773
Epoch:    8/30    Loss: 3.7897541459321977
Epoch:    8/30    Loss: 3.8204516021728514
Epoch:    8/30    Loss: 3.850849584007263
Epoch:    9/30    Loss: 3.755573563404451
Epoch:    9/30    Loss: 3.743623629570007
Epoch:    9/30    Loss: 3.781561588406563
Epoch:    9/30    Loss: 3.804346265554428
Epoch:    9/30    Loss: 3.837714919424057
Epoch:   10/30    Loss: 3.7379522537271153
Epoch:   10/30    Loss: 3.725995551991463
Epoch:   10/30    Loss: 3.77190391561985
Epoch:   10/30    Loss: 3.8005853183746336
Epoch:   10/30    Loss: 3.80991164252758
Epoch:   11/30    Loss: 3.7280719882039497
Epoch:   11/30    Loss: 3.730757861995697
Epoch:   11/30    Loss: 3.730751754760742
Epoch:   11/30    Loss: 3.7669393918275835
Epoch:   11/30    Loss: 3.809479929614067
Epoch:   12/30    Loss: 3.7231170450802775
Epoch:   12/30    Loss: 3.6949564624786375
Epoch:   12/30    Loss: 3.7364008557796478
Epoch:   12/30    Loss: 3.7831830166339873
Epoch:   12/30    Loss: 3.792822670507431
Epoch:   13/30    Loss: 3.7126853466033936
Epoch:   13/30    Loss: 3.6927947552442553
Epoch:   13/30    Loss: 3.720001023340225
Epoch:   13/30    Loss: 3.7577912693738935
Epoch:   13/30    Loss: 3.7872062445878982
Epoch:   14/30    Loss: 3.71201612866751
Epoch:   14/30    Loss: 3.682343319416046
Epoch:   14/30    Loss: 3.7344078679084776
Epoch:   14/30    Loss: 3.7443309134483336
Epoch:   14/30    Loss: 3.77489237575531
Epoch:   15/30    Loss: 3.7124079914376287
Epoch:   15/30    Loss: 3.6698835215091705
Epoch:   15/30    Loss: 3.73485600771904
Epoch:   15/30    Loss: 3.7619461012363433
Epoch:   15/30    Loss: 3.7600655731916426
Epoch:   16/30    Loss: 3.6972470316041934
Epoch:   16/30    Loss: 3.6862397878170015
Epoch:   16/30    Loss: 3.7206565790653228
Epoch:   16/30    Loss: 3.732853183078766
Epoch:   16/30    Loss: 3.7683126715183257
Epoch:   17/30    Loss: 3.678940508890152
Epoch:   17/30    Loss: 3.7088829630374907
Epoch:   17/30    Loss: 3.7285952334403993
Epoch:   17/30    Loss: 3.7750564227581025
Epoch:   18/30    Loss: 3.693602241259839
Epoch:   18/30    Loss: 3.6792489151000978
Epoch:   18/30    Loss: 3.7153882481098175
Epoch:   18/30    Loss: 3.74071560587883
Epoch:   18/30    Loss: 3.7408152737617493
Epoch:   19/30    Loss: 3.674694664710123
Epoch:   19/30    Loss: 3.677551249408722
Epoch:   19/30    Loss: 3.687728106832504
Epoch:   19/30    Loss: 3.734842054128647
Epoch:   19/30    Loss: 3.759466420459747
Epoch:   20/30    Loss: 3.686557890437345
Epoch:   20/30    Loss: 3.6655187238693236
Epoch:   20/30    Loss: 3.6934990501880645
Epoch:   20/30    Loss: 3.7300052551031113
Epoch:   20/30    Loss: 3.7417454318761827
Epoch:   21/30    Loss: 3.6675119099504423
Epoch:   21/30    Loss: 3.658510522890091
Epoch:   21/30    Loss: 3.6966787048101426
Epoch:   21/30    Loss: 3.734581877756119
Epoch:   21/30    Loss: 3.7333285326957704
Epoch:   22/30    Loss: 3.6571284011554885
Epoch:   22/30    Loss: 3.651191937923431
Epoch:   22/30    Loss: 3.7017444029808044
Epoch:   22/30    Loss: 3.7339160140752794
Epoch:   22/30    Loss: 3.743616231107712
Epoch:   23/30    Loss: 3.6642071420119238
Epoch:   23/30    Loss: 3.6521994450330735
Epoch:   23/30    Loss: 3.687728579592705
Epoch:   23/30    Loss: 3.715613380932808
Epoch:   23/30    Loss: 3.74017236597538
Epoch:   24/30    Loss: 3.663627580038997
Epoch:   24/30    Loss: 3.6552378009557724
Epoch:   24/30    Loss: 3.6810158603668213
Epoch:   24/30    Loss: 3.7092078464269638
Epoch:   24/30    Loss: 3.7275846886873247
Epoch:   25/30    Loss: 3.6780255919302984
Epoch:   25/30    Loss: 3.663547809791565
Epoch:   25/30    Loss: 3.6699489274978636
Epoch:   25/30    Loss: 3.7009117426395415
Epoch:   25/30    Loss: 3.715078913497925
Epoch:   26/30    Loss: 3.67279744978997
Epoch:   26/30    Loss: 3.6469233988285064
Epoch:   26/30    Loss: 3.6898121980667113
Epoch:   26/30    Loss: 3.699275132036209
Epoch:   26/30    Loss: 3.723770262670517
Epoch:   27/30    Loss: 3.636631201187588
Epoch:   27/30    Loss: 3.651474002313614
Epoch:   27/30    Loss: 3.683027679014206
Epoch:   27/30    Loss: 3.7139672506809234
Epoch:   27/30    Loss: 3.7295158178806305
Epoch:   28/30    Loss: 3.6505889506714087
Epoch:   28/30    Loss: 3.628259871149063
Epoch:   28/30    Loss: 3.6653973672389983
Epoch:   28/30    Loss: 3.7010521861314776
Epoch:   28/30    Loss: 3.728987779021263
Epoch:   29/30    Loss: 3.6513010133308454
Epoch:   29/30    Loss: 3.6313366602897643
Epoch:   29/30    Loss: 3.6751734742164612
Epoch:   29/30    Loss: 3.689155788421631
Epoch:   29/30    Loss: 3.7280489391088487
Epoch:   30/30    Loss: 3.652825957598738
Epoch:   30/30    Loss: 3.6188958399772644
Epoch:   30/30    Loss: 3.6709394254922865
Epoch:   30/30    Loss: 3.704297329521179
Epoch:   30/30    Loss: 3.7200978016614914
Model Trained and Saved
```

In [18]:
# Continue training for another 15 epochs

optimizer = torch.optim.Adam(trained_rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

with active_session():
    # training the model
    trained_rnn_2 = train_rnn(trained_rnn, batch_size, optimizer, criterion, 15, show_every_n_batches)

    # saving the trained model
    helper.save_model('./save/final_trained_rnn_2', trained_rnn_2)
    print('Model Trained and Saved')

Training for 15 epoch(s)...
Epoch:    1/15    Loss: 3.682072333264351
Epoch:    1/15    Loss: 3.730124739122391
Epoch:    1/15    Loss: 3.7494558283805848
Epoch:    1/15    Loss: 3.7421197113275526
Epoch:    1/15    Loss: 3.7502363882064818
Epoch:    2/15    Loss: 3.677651854843378
Epoch:    2/15    Loss: 3.6470519852399828
Epoch:    2/15    Loss: 3.6799519491434096
Epoch:    2/15    Loss: 3.708220761227608
Epoch:    2/15    Loss: 3.7350580753803255
Epoch:    3/15    Loss: 3.65728954605779
Epoch:    3/15    Loss: 3.652577089357376
Epoch:    3/15    Loss: 3.673100787115097
Epoch:    3/15    Loss: 3.7090265358924865
Epoch:    3/15    Loss: 3.7115169464826585
Epoch:    4/15    Loss: 3.6553947053212266
Epoch:    4/15    Loss: 3.642879879808426
Epoch:    4/15    Loss: 3.669261982655525
Epoch:    4/15    Loss: 3.697880366063118
Epoch:    4/15    Loss: 3.717589498281479
Epoch:    5/15    Loss: 3.650036093077402
Epoch:    5/15    Loss: 3.637693097472191
Epoch:    5/15    Loss: 3.66584221501350

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [12]:
# Continue training for another 15 epochs

trained_rnn_2 = helper.load_model('./save/final_trained_rnn_2')
optimizer = torch.optim.Adam(trained_rnn_2.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

with active_session():
    # training the model
    trained_rnn_3 = train_rnn(trained_rnn_2, batch_size, optimizer, criterion, 15, show_every_n_batches)

    # saving the trained model
    helper.save_model('./save/final_trained_rnn_3', trained_rnn_3)
    print('Model Trained and Saved')

Training for 15 epoch(s)...


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


Epoch:    1/15    Loss: 3.601941481256485
Epoch:    1/15    Loss: 3.6059847638607025
Epoch:    1/15    Loss: 3.606273383283615
Epoch:    1/15    Loss: 3.6040542474746706
Epoch:    1/15    Loss: 3.614287409257889
Epoch:    2/15    Loss: 3.5548399277797476
Epoch:    2/15    Loss: 3.5349178146123887
Epoch:    2/15    Loss: 3.550339245939255
Epoch:    2/15    Loss: 3.5680384600877764
Epoch:    2/15    Loss: 3.5723301954746245
Epoch:    3/15    Loss: 3.5173376584059093
Epoch:    3/15    Loss: 3.5099007538795473
Epoch:    3/15    Loss: 3.519442349243164
Epoch:    3/15    Loss: 3.5485923409938813
Epoch:    3/15    Loss: 3.564085441994667
Epoch:    4/15    Loss: 3.507384639640967
Epoch:    4/15    Loss: 3.5009533593177795
Epoch:    4/15    Loss: 3.529413005876541
Epoch:    4/15    Loss: 3.5361933893680573
Epoch:    4/15    Loss: 3.545175835466385
Epoch:    5/15    Loss: 3.4924470146285245
Epoch:    5/15    Loss: 3.4922484544992445
Epoch:    5/15    Loss: 3.5083582805871965
Epoch:    5/15    Lo

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:**
- **Sequence length**: At first, I began with `350` and `300` sequence length, but the model trained very slowly. Eventually, I found that decreasing it to `250` allowed the model to train and converge faster.
- **Hidden dim**: I set `hidden_dim = 256` to ensure that there were enough nodes/data for the final FC layer to predict the large number of possible word classes. 
- **Number of layers**: I set `num_layers = 2` as a deeper network is generally better. However, I did not increase it to 3 as I felt it might be too much for this task, and would also make the model much slower to train/increase the number of parameters.
- **Model architecture**:
    - Initially, I set **two** fully connected layers after the LSTM cells. However, I noticed that during training the model had problems converging, and trained very slowly. Thus, I decreased it to **1 fully connected layer** at the end.
- **Training parameters**:
    - I set a batch size of 32 (neither too big or too small)
    - I initially set the learning rate to be `0.001`. For the first ~30 epochs this performed pretty well, helping to bring my loss under 4.0. However, after that my loss kept oscillating and decreasing very slowly.
    - Thus, for the last 15 epochs I set learning rate to be `0.0005` (half the size) in case I had been taking too big steps. This helped to bring the loss under 3.5 quickly. 

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/final_trained_rnn_3')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [25]:
# run the cell multiple times to get different results!
gen_length = 300 # modify the length to your preference
prime_word = 'kramer' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


kramer:

jerry: what?

kramer:(putting his arm away) no, no!

kramer: oh.(jerry exits)

[setting: h&h bagel shop]

kramer: well, i'm heading back to my apartment and get some food back.

kramer:(to george) hey, you know, i think i have a little grace.

jerry: you know i like to sell a cadillac and i thought you were invited, you want a ride!(grabs the chip down)

kramer:(leaving) oh... yeah, well, i gotta get some ice out of my mind?

elaine: well, you know, we got a deal at the end of your pocket.

george: you know, you think something's going on to a library show.

kramer:(putting his picture on his shoulder)

kramer: oh, hey, hey, hey, hey. how's this guy?

jerry:(urgent) you know, i really like you, i gotta get going.

kramer: well, i got the tape.

morty:(yelling at the counter) what do you want? you gotta get hurt.

jerry: well, i gotta run up for a new long time. i got a feeling.

kramer:(entering monk's) hey, hey, how's lydia? i got a new hair button covered. he knows the next 

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [23]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.